In [47]:
# Please give me a  Python code, which will find a long call and long  Put for given set of stocks like["AAPL","VRAY","MSFT","MO"] for upto two week expiry.  
# The cost basis should be optimum. 

In [48]:
import yfinance as yf
from datetime import datetime, timedelta
import numpy as np

In [49]:
def find_options(stocks, expiry_period, price_difference):
    for stock in stocks:
        ticker = yf.Ticker(stock)

        # Get the current stock price
        current_price = ticker.history().tail(1)['Close'].iloc[0]

        # Find the closest expiration date within the desired period (up to two weeks)
        target_expiry = next((expiry for expiry in ticker.options if is_within_two_weeks(expiry, expiry_period)), None)
        
        if target_expiry:
            # Fetching call and put options for the selected expiry
            option_chain = ticker.option_chain(target_expiry)
            calls = option_chain.calls
            puts = option_chain.puts

            # Find a long call with strike price $5 away from current stock price
            long_call = calls.iloc[(calls['strike'] - (current_price + price_difference)).abs().idxmin()]

            # Find a long put with strike price $5 away from current stock price
            long_put = puts.iloc[(puts['strike'] - (current_price - price_difference)).abs().idxmin()]

            print(f"For {stock}: Long Call - {long_call['contractSymbol']}, Long Put - {long_put['contractSymbol']}")
        else:
            print(f"No options found for {stock} within the specified period.")

def is_within_two_weeks(expiry_date, expiry_period):
    # Convert the expiry_date to a datetime object and check if it's within the desired period
    expiry_date_dt = datetime.strptime(expiry_date, '%Y-%m-%d')
    return expiry_date_dt <= datetime.now() + timedelta(days=expiry_period)

stocks = ["RSG"]
expiry_period = 14 # Up to two weeks
price_difference = 5 # Dollar amount difference from the current price
find_options(stocks, expiry_period, price_difference)


For RSG: Long Call - RSG230818C00150000, Long Put - RSG230818P00140000


In [50]:
#  Greeks (Delta, Theta, Gamma, Vega) for Call option, using Black-Scholes model..
S = 147.27       # Current stock price
K = 150        # Call Option strike price
T = 0.025        # Time until expiration in years
r = 0.03       # Risk-free interest rate
sigma = 0.2    # Volatility
option_type = 'call' # Type of option, can be 'call' or 'put'
delta, theta, gamma, vega = black_scholes_greeks(S, K, T, r, sigma, option_type)
print(f"Delta: {delta}\nTheta: {theta}\nGamma: {gamma}\nVega: {vega}")

Delta: 0.2941477037510516
Theta: -33.36834243083672
Gamma: 0.07398923811978392
Vega: 0.08023560530339091


In [51]:
# Greeks (Delta, Theta, Gamma, Vega) for Put option, using Black-Scholes model.
S = 147.27       # Current stock price
K = 140        # Put Option strike price
T = 0.025        # Time until expiration in years
r = 0.03       # Risk-free interest rate
sigma = 0.2    # Volatility
option_type = 'put' # Type of option, can be 'call' or 'put'
delta, theta, gamma, vega = black_scholes_greeks(S, K, T, r, sigma, option_type)
print(f"Delta: {delta}\nTheta: {theta}\nGamma: {gamma}\nVega: {vega}")

Delta: -0.05045703291507807
Theta: -9.450247448190868
Gamma: 0.022307162786442278
Vega: 0.02419039247131931


In [52]:
# A Fourier transform can be used in pricing European options under certain mathematical models. 
#The Carr-Madan method is one example that utilizes Fast Fourier Transform (FFT) to compute option prices

In [53]:
# Call option price calculation based on Carr_Madan formula

def carr_madan(S, K, T, r, sigma, alpha, N=1000):
    # Characteristic function
    def phi(u):
        return np.exp(1j * u * (np.log(S) + (r - sigma ** 2 / 2) * T) - sigma ** 2 * u ** 2 * T / 2)

    # Carr-Madan integral
    delta_u = 0.01  # Step size
    integral = 0.0
    for u in np.arange(0, N, delta_u):
        integral += np.real(np.exp(-1j * u * np.log(K)) * phi(u - (alpha + 1) * 1j))

    # Call price
    call_price = np.exp(-alpha * np.log(K)) * integral * delta_u / np.pi

    return call_price

# Parameters
S = 147.27
K = 150
T = 0.025
r = 0.03
sigma = 0.2
alpha = 1.5
# Call option price calculation
call_price = carr_madan(S, K, T, r, sigma, alpha)
print(f"The call option price: {call_price}")

The call option price: 1606.1449661677034


In [54]:
# Put option price calculation based on Carr_Madan formula

def carr_madan_put(S, K, T, r, sigma, alpha, N):
    # Characteristic function
    def phi(u):
        return np.exp(1j * u * (np.log(S) + (r - sigma ** 2 / 2) * T) - sigma ** 2 * u ** 2 * T / 2)

    # Carr-Madan integral
    delta_u = 0.01  # Step size
    integral = 0.0
    for u in np.arange(0, N, delta_u):
        integral += np.real(np.exp(-1j * u * np.log(K)) * phi(u - (alpha + 1) * 1j))

    # Call price
    call_price = np.exp(-alpha * np.log(K)) * integral * delta_u / np.pi

    # Put price using put-call parity
    put_price = call_price - S + K * np.exp(-r * T)

    return put_price

S = 147.27        # Current stock price
K = 140     # Put Option strike price
T = 0.025        # Time until expiration in years
r = 0.03       # Risk-free interest rate
sigma = 0.2    # Volatility
alpha = 1.5    # Damping factor
N = 1000

put_price = carr_madan_put(S, K, T, r, sigma, alpha, N)
print(f"The put option price is: {put_price}")

The put option price is: 477.04971046193737
